TO-DO:
- add merge_geom parameter to dissolve to automatically pull that using the created relation
- add cut and interpolate aggregators

In [1]:
import linref as lr
import linref_011 as lr011
from linref.events import EventsData
from linref.events import common, selection, modify, utility, analyze, geometry
import numpy as np
import pandas as pd
from shapely.geometry import LineString
import scipy.sparse as sparse
import geopandas as gpd
import linref as lr
import warnings

In [2]:
lr.set_default_lrs(key_col=['county', 'route'], loc_col='loc', beg_col='beg', end_col='end', geom_col='geometry', closed='left_mod')

df = gpd.read_file('testing/data/linear-events-3.geojson').set_index('index')

In [3]:
df.lr.add_chaining(inplace=True)
df.lr.active_lrs

LRS(key_col=['county', 'route', 'chain'], loc_col=loc, beg_col=beg, end_col=end, geom_col=geometry, closed=left_mod)

In [4]:
dissolved, relation = df.lr.dissolve(return_relation=True)

In [5]:
df.lr.add_geom_m(inplace=True)

In [6]:
relation.linemerge_m(data=df.geometry_m)[0]

LINESTRING M (0.0 0.0 0.0, 0.8 0.0 0.4) # linref compatibility approximation

In [14]:
relation['geometry_m'].linemerge_m()

0    LINESTRING M (0.0 0.0 0.0, 0.8 0.0 0.4) # linr...
1    LINESTRING M (1.0 0.0 0.5, 1.0 1.0 1.1, 2.0 1....
2    LINESTRING M (10.0 0.0 10.0, 12.0 0.0 12.0, 13...
3    LINESTRING M (13.0 0.0 13.0, 13.0 1.0 13.1, 14...
4    LINESTRING M (1.0 3.0 1.8, 1.0 4.0 3.0) # linr...
dtype: object

In [12]:
relation[['volume', 'class']].mode()

array([[100, 'B'],
       [200, 'A'],
       [400, 'A'],
       [300, 'B'],
       [800, 'A']], dtype=object)